## Sélection des vidéos d'intérêt (pour analyse ultérieure)
#### Sélection sur mots ou séquences clés
#### Sélection sur similarité avec un texte de référence (basée sur comptages simples ou tfidf)

##### Installation librairies et modules tiers

In [ ]:
# Installations librairies tiers ici inutiles
#!pip install spacy-langdetect
#!pip install spacy-lefff
#!pip install pyldavis
#!pip install swifter

In [ ]:
# mais le modèle moyen français de Spacy l'est, car il faut calculer les lemmes du texte de référence
!pip install https://github.com/explosion/spacy-models/releases/download/fr_core_news_md-2.3.0/fr_core_news_md-2.3.0.tar.gz
!pip install fr_core_news_md

     |████████████████████████████████| 46.1MB 98kB/s 
     |████████████████████████████████| 10.4MB 3.6MB/s 
     |████████████████████████████████| 1.1MB 50.9MB/s 
  Created wheel for fr-core-news-md: filename=fr_core_news_md-2.3.0-cp36-none-any.whl size=46100671 sha256=35301c50815d741000d4f47b33929a63c133d02a9a1808bf605f450d9c4ebcf7
  Stored in directory: /root/.cache/pip/wheels/5b/dc/d5/f9cf6ba6cb8094a22f0aefab2cb654547c5584ce08077cbdbb
Successfully built fr-core-news-md
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


## Ensuite, REDEMARRER L'ENVIRONNEMENT D'EXECUTION

### Librairies, variables globales...¶

In [ ]:
# 1.a
# librairies générales (on ne se servira pas forcément de toutes)
import os
import csv
import re
import math
#import pickle
import datetime as dt
from ast import literal_eval

In [ ]:
# 1.b
# librairies générales data science
import numpy as np
import pandas as pd
# import swifter
# stats
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from scipy.stats import kendalltau

In [ ]:
# 1.c
# pour affichage dtypes pandas quand plus de 60 colonnes
pd.set_option('display.max_rows', 120)

In [ ]:
# 1.d
# la librairie graphique la plus simple d'utilisation
import plotly.express as px
import plotly.graph_objects as go

#### librairies text-mining / NLP

In [ ]:
# 1.e
import spacy
nlp = spacy.load("fr_core_news_md")
# on ajoute à la pipeline standard de spacy le détecteur de langue
# from spacy_langdetect import LanguageDetector
# nlp.add_pipe(LanguageDetector(), name='language_detector', last=True)

In [ ]:
# 1.f
import gensim
from gensim import corpora

In [ ]:
# 1.h imports spécifiques à google colab
from google.colab import files, drive

#### Autres fichiers

In [ ]:
# 1.i
SELECTION_MOTS_COURANTS = "sélection_mots_courants.txt"

### Lecture du fichier de 1300 vidéos prétraitées
##### Prétraitées : on a effectué au préalable l'analyse nlp de Spacy sur les sous_titres et renseigné :

###### 'langue' langue de la vidéo tel que détecté par spacy (on garde 'fr')
###### 'tkn_sous_titres' listes de token avec un tuple par token, contenant en : 
0 : texte, 1 : index dans doc, 2 : position dans doc, 3 : lemme, 4 : pos, 5 : index dans doc de la tête, 6 : est un mot stop, 7 : type entité nommée (si existe)
###### 'ent_sous_titres' liste d'entités nommées avec un tuple par entité nommée, contenant en :
0 : texte, 1 : position du début dans doc, 2 : position fin, 3 : type entité nommée

In [ ]:
# 2.a passage obligé pour récupérer un fichier sur le drive
# il va falloir choisir le drive (account Google) puis renseigner un mot de passe généré
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 2.b
chemin_nom_dataframe = "/content/drive/My Drive/Datasets NLP/COVID 19/videos_covid_nlp.csv"
df_videos = pd.read_csv(chemin_nom_dataframe, sep=',', encoding='utf-8')

In [ ]:
# 2.c
# étape de retransformation des chaînes analyse de résultats sauvegardés en liste de tuples
df_videos['tkn_sous_titres'] = df_videos['tkn_sous_titres'].map(lambda x: literal_eval(str(x)))
df_videos['ent_sous_titres'] = df_videos['ent_sous_titres'].map(lambda x: literal_eval(str(x)))

In [ ]:
# 2.d
# nombre de vidéos, liste des colonnes (champs) et leurs types
nb_videos = len(df_videos)
print(nb_videos)
df_videos.dtypes

1342


video_id               object
channel_id             object
titre                  object
description            object
date_publication       object
durée                  object
catégorie_YT          float64
tags                   object
nb_vues               float64
nom_chaîne             object
sous_titres            object
nb_mots               float64
mode_création          object
nb_vues_chaîne          int64
catégorie_SP           object
langue_sous_titres     object
tkn_sous_titres        object
ent_sous_titres        object
dtype: object

In [ ]:
print(df_videos.iloc[0]['tkn_sous_titres'])

[('Dans', 0, 0, 'dans', 'ADP', 2, True, 0), ('le', 1, 5, 'le', 'DET', 2, True, 0), ('monde', 2, 8, 'monde', 'NOUN', 8, False, 0), ('du', 3, 14, 'de', 'DET', 4, True, 0), ('pétrole', 4, 17, 'pétrole', 'NOUN', 2, False, 0), ('c’', 5, 25, 'c’', 'PRON', 8, True, 0), ('est', 6, 27, 'être', 'AUX', 8, True, 0), ('un', 7, 31, 'un', 'DET', 8, True, 0), ('événement', 8, 34, 'événement', 'NOUN', 8, False, 0), ('\xa0', 9, 43, '  ', 'SPACE', 8, False, 0), (':', 10, 44, ':', 'PUNCT', 8, False, 0), ('en', 11, 46, 'en', 'ADP', 12, True, 0), ('juin', 12, 49, 'juin', 'NOUN', 8, False, 0), ('2020', 13, 54, '2020', 'NUM', 12, False, 0), ('la', 14, 59, 'le', 'DET', 15, True, 0), ('compagnie', 15, 62, 'compagnie', 'NOUN', 18, False, 0), ('BP', 16, 72, 'BP', 'PROPN', 15, False, 383), ('a', 17, 75, 'avoir', 'AUX', 18, True, 0), ('annoncé', 18, 77, 'annoncer', 'VERB', 8, False, 0), ('10', 19, 85, '10', 'DET', 20, False, 0), ('000', 20, 88, '000', 'DET', 21, False, 0), ('suppressions', 21, 92, 'suppression', 'N

#### Cette structure est assez complète, mais ce dont on a ici besoin, c'est une simple liste des lemmes, dont la sélection se base sur les parties du discours pleines, et on élimine les stop words

##### Fonction qui à partir des données lemme, POS, et stop-word renvoie le lemme ou rien du tout si non retenu

In [ ]:
# 2.e
# répartition un peu arbitraire entre catégories vides et pleines
# les premières faisant office de stopwords, dans un modèle sacs de mots
CATEGORIES_VIDES = ['ADP', 'AUX', 'CONJ', 'CCONJ', 'DET', 'PART', 'PRON',
                    'PUNCT', 'SCONJ', 'SYM', 'SPACE']
CATEGORIES_PLEINES = ['ADJ', 'ADV', 'INTJ', 'NOUN', 'NUM', 'PROPN', 'VERB', 'X']

In [ ]:
# 2.f
# cette fonction servira à la fois sur les analyses déjà faites et sur le texte de référence utilisé à fins de comparaisons
def selectionne_lemme(lemme, pos, est_stopword):
  selection_lemme = ''
  if pos in CATEGORIES_PLEINES and est_stopword == False:
    selection_lemme = lemme
  return selection_lemme


In [ ]:
# 2.g
# utilisation sur les analyses déjà faites, création d'une nouvelle colonne, en utilisant la fonction suivante,
# qui prend en argument la liste des tokens résultat de l'analyse nlp spacy effectuée auparavant
def cree_l_lemmes_sur_resultat_nlp(l_tokens_nlp):
  l_lemmes = []
  for token in l_tokens_nlp:
    lemme = selectionne_lemme(token[3], token[4], token[6])
    if lemme != '':
      l_lemmes.append(lemme)
  return l_lemmes


In [ ]:
# 2.h
# appliquer ceci en créant une nouvelle colonne du dataframe stockant la liste des lemmes d'intérêt
df_videos['lemmes_sous_titres'] = df_videos['tkn_sous_titres'].apply(lambda x: cree_l_lemmes_sur_resultat_nlp(x))

In [ ]:
print(df_videos.iloc[0]['lemmes_sous_titres'])

['monde', 'pétrole', 'événement', 'juin', '2020', 'compagnie', 'BP', 'annoncer', 'suppression', 'emploi', '13', '175', 'milliard', 'dollar', 'dépréciation', 'actif', 'conséquence', 'coronavirus', 'porter', 'coup', 'arrêt', 'économie', 'mondial', 'demande', 'pétrole', 'nouveau', 'perspective', 'favorable', 'prévoir', 'interroger', 'coronavirus', 'sonne', 'fin', 'pétrole', 'BP', 'grand', 'compagnie', 'pétrolier', 'monde', 'annoncer', 'mesure', 'drastique', 'falloir', 'lire', 'ligne', 'communiqué', 'annoncer', 'BP', 'prévoir', 'pandémie', 'impact', 'durable', 'économie', 'mondial', 'demande', 'énergie', 'potentiellement', 'faible', 'période', 'prolonger', 'reprise', 'BP', 'changement', 'long', 'terme', 'changement', 'prix', 'multinationale', 'cours', 'moyen', 'baril', 'pétrole', '-', '-', 'brer', 'servir', 'référence', 'devoir', 'chuter', 'prochain', 'année', 'fixer', 'autour', '55', 'dollar', '2021', '2050', 'nettement', '70', 'dollar', 'anticiper', 'précédemment', 'prévision', 'pétrole'

### Recherche sur mots-clés

#### On va d'une part toper (1 ou 0) les vidéos qui présentent la ou les combinaisons de lemmes clés dans les sous titres
On aurait pu aussi considérer les descriptions et les tags, mais dès lors que les combinaisons d'intérêt sont dans les sous-titres, elles le sont aussi dans les descriptions et tags 

#### Et d'autre part mesurer la force des mots clés (la fréquence relative dans les sous-titres)

##### On repère cela à partir de sous-séquences à retrouver dans la liste des lemmes

In [ ]:
# 3.a
SOUS_SEQUENCES_CLE = [['masque', 'chirurgical'], ['masque'], ['stock'], ['laboratoire'],
                     ['gel', 'hydroalcoolique'], ['hydroalcoolique'], ['hydroxychloroquine'], ['chloroquine'],
                     ['didier', 'raoult'], ['raoult'], ['commission'], ['commission', 'enquête']]
                     

##### Fonctions de recherche de sous-séquences

In [ ]:
# 3.c
# fonction copiée depuis stackoverflow
# on va chercher une sous-séquence dans celle du texte éclaté en mots
def index_subseq_in_seq(subseq, seq):
    
    i, n, m = -1, len(seq), len(subseq)
    try:
        while True:
            i = seq.index(subseq[0], i + 1, n - m + 1)
            if subseq == seq[i:i + m]:
               return i
    except ValueError:
        return -1

In [ ]:
# 3.d
# on peut aussi compter le nombre de séquences
def nb_subseq_in_seq(subseq, seq):
    
    nb_subseq = 0
    remaining_seq = seq
    while len(remaining_seq) >= len(subseq):
        index = index_subseq_in_seq(subseq, remaining_seq)
        if index >= 0:
            remaining_seq = remaining_seq[index + len(subseq):]
            nb_subseq += 1
        else:
            remaining_seq = []
    
    return nb_subseq

##### on créer deux colonnes, une indiquant présence ou non d'une des sous-séquences à détecter, et l'autre fréquence relative de l'ensemble des sous-séquences à rechercher


In [ ]:
# 3.e
def presence_sous_sequences(liste_tokens, liste_sous_sequences):
    presence = 0
    for sous_sequence in liste_sous_sequences:
        if index_subseq_in_seq(sous_sequence, liste_tokens) >= 0:
            presence = 1
            break
    return presence  

In [ ]:
# 3.f
def lit_presence_termes_cle(l):
    global compteur
    presence_termes_cle = 0
    liste_tokens = l['lemmes_sous_titres']
    if isinstance(liste_tokens, list):
        presence_termes_cle = presence_sous_sequences(liste_tokens, SOUS_SEQUENCES_CLE)
        #print("%05d" % (compteur), end="\r")
        compteur += 1
    return presence_termes_cle

In [ ]:
# 3.g
compteur = 0
df_videos['presence_termes_cle'] = df_videos.apply(lambda l: lit_presence_termes_cle(l), axis=1)

In [ ]:
# 3.h
def freq_relative_sous_sequences(liste_tokens, liste_sous_sequences):
    freq_relative = 0.0
    nb_tokens = len(liste_tokens)
    nb_presences = 0
    for sous_sequence in liste_sous_sequences:
        nb_presences += nb_subseq_in_seq(sous_sequence, liste_tokens)
    if nb_tokens > 0:
        freq_relative = 100 * nb_presences / nb_tokens
    return freq_relative

In [ ]:
# 3.i
def calcule_freq_relative_termes_cle(l):
    global compteur
    freq_relative_termes_cle = 0
    liste_tokens = l['lemmes_sous_titres']
    if isinstance(liste_tokens, list):
        freq_relative_termes_cle = freq_relative_sous_sequences(liste_tokens, SOUS_SEQUENCES_CLE)
        #print("%05d" % (compteur), end="\r")
        compteur += 1
    return freq_relative_termes_cle

In [ ]:
# 3.j
compteur = 0
df_videos['freq_relative_termes_cle'] = df_videos.apply(lambda l: calcule_freq_relative_termes_cle(l), axis=1)

In [ ]:
# 3.k
df_videos_termes_cle = df_videos.loc[df_videos['presence_termes_cle']==1]
print(len(df_videos_termes_cle))

699


In [ ]:
# 3.l
df_videos_termes_cle.head()

,video_id,channel_id,titre,description,date_publication,durée,catégorie_YT,tags,nb_vues,nom_chaîne,sous_titres,nb_mots,mode_création,nb_vues_chaîne,catégorie_SP,langue_sous_titres,tkn_sous_titres,ent_sous_titres,lemmes_sous_titres,presence_termes_cle,freq_relative_termes_cle
6,91v0G_v2VYs,UCckz6n8QccTd6K_xdwKqa0A,Édouard Philippe masqué devant l'Assemblée nat...,"""Les députés et les ministres devront porter u...",2020-06-23,PT46S,25.0,NaN,815.0,AFP,députés les ministres devront porter un masque...,49.0,generated,394905244,Médias centraux,fr,"[(députés, 0, 0, député, NOUN, 0, False, 0), (...","[(francis vercamer d monsieur, 159, 186, PER)]","[député, ministre, devoir, porter, masque, pre...",1,8.333333
9,FogBL3V4UBg,UCckz6n8QccTd6K_xdwKqa0A,Première vague de chaleur post-déconfinement e...,Réservation obligatoire et distanciation physi...,2020-06-26,PT1M36S,25.0,NaN,277.0,AFP,une arme en les mains ou utiliser une solution...,114.0,generated,394905244,Médias centraux,fr,"[(une, 0, 0, un, DET, 1, True, 0), (arme, 1, 4...","[(le soleil, 249, 258, LOC), (ballon rond, 377...","[arme, main, utiliser, solution, hydroalcooliq...",1,2.272727
12,KZX5S-9ZIJs,UCckz6n8QccTd6K_xdwKqa0A,"REPLAY - Macron: ""Nous sommes en guerre sanita...","""Nous sommes en guerre sanitaire"" a déclaré lu...",2020-03-16,PT20M59S,25.0,NaN,NaN,AFP,françaises français mes chers compatriotes jeu...,2637.0,generated,394905244,Médias centraux,fr,"[(françaises, 0, 0, français, ADJ, 0, False, 0...","[(covin, 163, 168, LOC), (la france, 684, 693,...","[français, français, compatriote, jeudi, soir,...",1,0.270758
13,LnpXtFstLAw,UCckz6n8QccTd6K_xdwKqa0A,REPLAY - Covid-19: Macron promet des décisions...,En déplacement en Corse Emmanuel Macron a indi...,2020-09-10,PT36M37S,25.0,['AFP' 'agence france presse' 'news' 'info' 'a...,159.0,AFP,bonjour à toutes et tous j'enlève le masque pa...,5947.0,b_generated,394905244,Médias centraux,fr,"[(bonjour, 0, 0, bonjour, NOUN, 0, False, 0), ...","[(mesdames messieurs, 131, 149, PER), (corse, ...","[bonjour, enlever, masque, être, bon, distance...",1,0.123153
14,LrtPbVRncnQ,UCckz6n8QccTd6K_xdwKqa0A,Quelques centaines de manifestants anti-masque...,"""L’imposer maintenant je ne trouve pas ça très...",2020-08-29,PT1M43S,25.0,['FRANCE' 'SANTÉ' 'VIRUS' 'ÉPIDÉMIE' 'MANIFEST...,639.0,AFP,des lois sont injustes pour quelles raisons au...,211.0,b_generated,394905244,Médias centraux,fr,"[(des, 0, 0, un, DET, 1, True, 0), (lois, 1, 4...","[(ça allume la lampe, 913, 931, MISC)]","[loi, injuste, raison, virus, létal, personne,...",1,7.936508


##### Aperçu rapide sur quelques vidéos, choisir quelques valeurs de idx dans exécutions de la cellule suivante

In [ ]:
# 3.m
idx = 234
l_tokens = df_videos_termes_cle.iloc[idx]['lemmes_sous_titres']
print(l_tokens)
print(len(l_tokens))
print('----------------------')
presence = df_videos_termes_cle.iloc[idx]['presence_termes_cle']
freq_relative = df_videos_termes_cle.iloc[idx]['freq_relative_termes_cle']
print("présence: %d - fréq. relative %.2f" % (presence, freq_relative))

['être', 'centre', 'médical', 'site', 'veolia', 'changy', 'hier', 'dizaine', 'infirmier', 'prendre', 'possession', 'local', 'objectif', 'tester', '280', 'salarié', 'entreprise', 'découverte', 'cas', 'comics', '19', 'employé', 'passage', 'dépister', 'bout', 'forcément', 'spécialement', 'inquiet', 'matériel', 'masque', 'matériel', 'désinfecter', 'camion', 'gant', 'spécialement', 'inquiet', 'bon', 'rassurant', 'faire', 'test', 'salarié', 'contaminé', 'contracter', 'virus', 'obsèque', '6', 'juin', 'diagnostic', 'mise', 'isolément', 'collègue', 'tester', 'positif', 'travailler', 'étage', 'bâtiment', 'falloir', 'agir', 'vite', 'stopper', 'chaîne', 'transmission', 'travailler', 'faire', 'cercle', 'contact', 'épisode', 'identifier', 'ensemble', 'personnel', 'test', 'généraliter', 'entreprise', 'proposer', 'ensemble', 'collaborateur', 'rappeler', 'cas', 'avérer', 'prendre', 'décision', 'alerter', 'rs', 'fois', 'prélèvement', 'effectuer', 'employé', 'répondre', 'question', 'identifier', 'éventue

### Pour gensim, utilisé tant pour la similarité que pour l'analyse de topics
#### on va créer la matrice (document, terme) des sous-titres (plus l'équivalent pour le texte de référence)
##### Gensim utilise des dictionnaires associant chaînes de caractères à index en interne, d'où travailler sur les lemmes

#### Mais auparavant, récupération du texte de référence (dépendant du sujet de référence)

In [ ]:
# 4.a
chemin_nom_reference = "/content/drive/My Drive/Datasets NLP/COVID 19/texte_reference_controverse_coronavirus.txt"

In [ ]:
# 4.b
fichier_ref = open(chemin_nom_reference, 'r', encoding='utf8')
texte_ref = fichier_ref.read()
fichier_ref.close()

In [ ]:
print(texte_ref)

La pandémie de Covid-19 en France est une crise sanitaire majeure provoquée par une maladie infectieuse émergente apparue fin 2019 en Chine continentale, la maladie à coronavirus 2019 (Covid-19), dont l'agent pathogène est le SARS-CoV-2. Ce virus est à l'origine d'une pandémie, déclarée le 11 mars 2020 par l'Organisation mondiale de la santé.

Lorsque la pandémie arrive en Europe, la France est touchée comme d'autres pays voisins même si l'Allemagne est plus résiliente.

En France, elle est associée à une surmortalité journalière de 33 % entre le 1er mars et le 30 avril 2020 comparée aux valeurs moyennes des années 2000 à 2019 : « 2 120 décès ont été recensés en moyenne chaque jour par l'état civil, contre 1 590 en moyenne entre 2000 et 2019 ».

Le 24 janvier 2020, trois premiers cas sont recensés sur le territoire métropolitain. À la mi-mars 2020, afin de stopper la diffusion exponentielle du coronavirus et pour réduire le plus possible le nombre de personnes atteintes et donc de mort

###### procéder à l'analyse syntaxique du texte de référence

In [ ]:
# 4.d
# construction d'une liste de lemmes pour analyses sacs de mots
# l'analyse linguistique a déjà été faite par spacy, on en passe le résultat (tokens_spacy)
def cree_l_lemmes_sur_resultat_brut_spacy(tokens_spacy):

  l_lemmes = []  
  for token in tokens_spacy:
    lemme = selectionne_lemme(token.lemma_, token.pos_, token.is_stop)
    if lemme != '':
      l_lemmes.append(lemme)
  return l_lemmes

In [ ]:
# 4.e
# performer l'analyse syntaxique spacy et en déduire le sac de lemmes
tokens_nlp_ref = nlp(texte_ref)
l_tokens_ref_normes = cree_l_lemmes_sur_resultat_brut_spacy(tokens_nlp_ref)
# besoin d'une "anti-normalisation" pour aligner le texte sur les caractéristiques des sous-titres
# s'aligner sur la mauvaise qualité des sous-titres, et passer tout en minuscule
# (mais pas les sigles, à voir)
# et éliminer les symboles '—' et '«' (il n'y a pas la fermante)
l_tokens_ref = []
for token_norme in l_tokens_ref_normes:
    token = token_norme.lower()
    if token not in ['—', '«']:
        l_tokens_ref.append(token)

##### opérations de constructions d'objets à gensim

In [ ]:
# 4.f
# parcourir l'ensemble du dataframe pour obtenir les données de la matrice document / termes
# sous la forme d'une liste par documents de vecteurs de termes 
# on stocke une liste de listes de termes (ici les lemmes), vidéo par vidéo
mat_doc_tokens_ss_titres = []

# parcourir les vidéos
for ix_video, l_tokens in df_videos['lemmes_sous_titres'].iteritems():
    
    mat_doc_tokens_ss_titres.append(l_tokens)
    #print("%05d" % (ix_video), end="\r")
    
# rajouter en dernier le texte de référence
mat_doc_tokens_ss_titres.append(l_tokens_ref)

# création d'un dictionnaire donnant le terme à partir de son identifiant numérique
dico_termes_ss_titres = corpora.Dictionary(mat_doc_tokens_ss_titres)

# conversion en une liste de vecteurs fréquence des termes sur documents 
# représentés sous la forme (id numérique du terme, fréquence du terme dans le document)
# les fréquences à 0 n'y sont pas présentes
mat_doc_termes_freq_ss_titres = [dico_termes_ss_titres.doc2bow(l_tokens) \
                                 for l_tokens in mat_doc_tokens_ss_titres]

In [ ]:
# 4.g
# examiner les objets créés (et stocker taille de corpus et taille de dico)
taille_corpus = len(mat_doc_tokens_ss_titres)
print("taille du corpus : %d textes" % (taille_corpus))
# vaut le nombre de vidéos dans l'échantillon, plus 1, le texte de référence

# conserver aussi le nombre de vidéos avec présence du terme clé
taille_corpus_avec_termes_cle = len(df_videos_termes_cle)
print("dont vidéos avec termes clé : %d textes" % \
      (taille_corpus_avec_termes_cle))

# prendre deux trois indices (numéros de documents)
idx = 100
print("tokens pour document %d :" % (idx), mat_doc_tokens_ss_titres[idx])
print("termes et fréquences pour document %d :" % (idx), mat_doc_termes_freq_ss_titres[idx])
taille_dico = len(dico_termes_ss_titres)
print("dico termes ss titres : %d termes dans le dictionnaire" % (taille_dico))
for i in range(1000,1010): print(i, dico_termes_ss_titres[i])

taille du corpus : 1343 textes
dont vidéos avec termes clé : 699 textes
tokens pour document 100 : ['union', 'européen', 'limiter', 'exportation', 'masquer', 'équipement', 'médical', 'protection', 'garantir', 'propre', 'approvisionnement', 'face', 'pandémie', 'devoir', 'coronavirus', 'gerald', 'cardiaque', 'médecin', 'urgentiste', 'bonsoir', 'réagir', 'direction', 'arte', 'france', 'chiffre', 'venir', 'tomber', 'record', 'décès', 'italie', '24', 'heure', '368', 'décès', '24', 'heure', 'record', 'jour', 'réaction', 'rapport', 'chiffre', 'voir', 'épidémie', 'progresse', 'italie', 'mesure', 'confinement', 'falloir', 'regarder', 'détail', 'décéder', 'fois', 'donner', 'donnée', 'brut', 'décès', 'savoir', 'france', 'jour', 'koror', 'coronavirus', 'sûr', 'yer', '1700', 'décès', 'jour', 'falloir', 'aller', 'recontextualiser', 'voir', 'nom', 'nombre', 'décès', 'augmente', 'revanche', 'italie', 'mesure', 'confinement', 'vouloir', 'système', 'santé', 'peut-être', 'saturation', 'engager', 'aller',

#### La simple mesure des fréquences brutes de termes dans les documents, assortis à une bonne mesure comme celle du cosinus qui permet de normaliser la taille des documents, permet déjà d'examiner les documents par rapport à leur similarité avec le texte de référence

#### On va examiner les similarités en étudiant :
1) le comptage simple<p>
2) le tf-idf (en essayant quelques variantes)<p>
3) pour mémoire, le one-hot encoding<p>

#### Mesurer la similarité des documents avec le texte de référence passe par :
- transformer les listes de termes, fréquence (ou poids de façon plus générale) en matrices (creuses)<p>
- calculer le cosinus entre deux documents<p>
- représenter graphiquement ces similarités en les mettant en regard avec les résultats de l'approche par mots clé<p>

#### Utilitaires : 
##### vecteurs creux : transformer une liste de (id, poids) avec les seuls id effectivement présents dans le document, en un np.array de longueur le nombre de termes dans le dictionnaire, pour éviter de calculer en bloc la matrice de similarité
#### Et calculer le cosinus entre deux vecteurs

In [ ]:
# 5.a
# A partir d'un vecteur dense de (index, poids) issu de la construction gensim
# poids créer un vecteur sur l'ensemble des composantes du dictionnaire, donc creux (sparse)
def vect_gensim_2_vect_creux_np(vect_poids, taille_dico):
    vect_creux_np = np.zeros(taille_dico)
    for (index, poids) in vect_poids:
        vect_creux_np[index] = poids
    return vect_creux_np

In [ ]:
# 5.b
def similarite_cosinus(vect_np_1, vect_np_2):
    cosinus = np.dot(vect_np_1, vect_np_2) / \
              (np.sqrt(np.dot(vect_np_1,vect_np_1)) * np.sqrt(np.dot(vect_np_2,vect_np_2)))
    return cosinus

##### Fonction pour calculer les similarités des sous-titres avec le texte de référence, et ensuite les présenter triées

In [ ]:
# 5.c
# pour le calcul des similarités :
# on étend les vecteurs de poids (fréquence, etc.) sur la taille du dictionnaire (10000+ termes)
# et on calcule le cosinus sur le produit scalaire entre le vecteur de référence
# et le vecteur de sous-titres couramment considéré

# pour chaque (sous-titres de) vidéos on stocke un triplet de similarités :
# présence (fréquence absolue) du terme clé, fréquence relative et surtout similarité vectorielle avec le texte de référence
# similarité basée, similarité qui peut être de fréquence absolue, tf-idf, voire one-hot encoding

def calcule_similarites_avec_ref(mat_doc_termes_poids, taille_corpus, taille_dico, df_videos):
    l_similarites = []

    # créer le vecteur sparse pour le texte de référence, qui est en dernier (cas sparse) - en numpy
    vect_creux_ref = vect_gensim_2_vect_creux_np(mat_doc_termes_poids[taille_corpus-1], taille_dico)
                                                 
    # boucler sur les autres vecteurs du corpus et calculer la similarité de cosinus avec le texte de référence
    # et stocker des triplets :
    # (index_video, présence termes clé, fréquence relative termes clé, similarité avec texte de référence)
    # index vidéo pour garder une référence de la vidéo d'origine quand la liste aura été triée
    
    for idx_video, vect_doc_termes_poids in enumerate(mat_doc_termes_poids[:taille_corpus-1]):
        
        # lire les éléments relatifs aux termes clés
        presence_termes_cle = df_videos.iloc[idx_video]['presence_termes_cle']
        freq_relative_termes_cle = df_videos.iloc[idx_video]['freq_relative_termes_cle']
        
        # calculer la similarité cosinus
        vect_creux_compare = vect_gensim_2_vect_creux_np(vect_doc_termes_poids, taille_dico)
        similarite = similarite_cosinus(vect_creux_compare, vect_creux_ref)
                 
        l_similarites.append((idx_video, presence_termes_cle, freq_relative_termes_cle, similarite))

    # trier par similarité, les plus fortes en tête
    l_similarites.sort(key=lambda x: x[3], reverse=True)
    
    return l_similarites

In [ ]:
# 5.d
# sur les vidéos triées par ordre de similarité décroissant, noter les "premières" vidéos
# s'il y a N vidéos taggées explicitement des mots clés, les afficher toutes
# et éventuellement compléter de telle sorte qu'il y ait au moins N vidéos en tout
# on va lister des quadruplets (index_video, presence_termes_cle, freq_relative_termes_cle, similarité)

def liste_videos_similaires(l_similarites_triees, nb_videos_presence_explicite):
    l_videos_similaires = []
    idx_video_presence_explicite = 0
    idx_video_presence_non_explicite = 0
    for no_ordre, (idx_video, presence_termes_cle, freq_relative_termes_cle, similarite) \
        in enumerate(l_similarites_triees):
        video_info_similarite = (idx_video, presence_termes_cle, freq_relative_termes_cle, similarite)
        if idx_video_presence_explicite < nb_videos_presence_explicite:
            l_videos_similaires.append(video_info_similarite)
            if presence_termes_cle >= 1:
                idx_video_presence_explicite += 1
            else:
                idx_video_presence_non_explicite += 1
        else:
            if idx_video_presence_non_explicite < nb_videos_presence_explicite:
                l_videos_similaires.append(video_info_similarite)
                idx_video_presence_non_explicite += 1
        if idx_video_presence_explicite >= nb_videos_presence_explicite: 
            break
    return l_videos_similaires

In [ ]:
# 5.e
# montre la liste des premières vidéos
def affiche_videos_similaires(l_similarites, nb_videos_presence_explicite):
    l_videos_similaires = liste_videos_similaires(l_similarites, nb_videos_presence_explicite)
    for (idx_video, presence_termes_cle, freq_relative_termes_cle, similarite) in l_videos_similaires:
        print("vidéo %d\t fréq. relative termes clé : %.2f\t similarité : %.2f" % \
              (idx_video, freq_relative_termes_cle, similarite))
    

In [ ]:
# 5.f
# représenter graphiquement la distribution des similarités (sur toutes les videos)

def montre_repartition_similarites(l_similarites_triees, nature_similarite):
    
    # on a besoin de similarité avec présence mot clé explicite ou non
    similarite_explicite = [similarite for (idx_video, presence_termes_cle, freq_relative_termes_cle, similarite) \
                            in l_similarites_triees if presence_termes_cle >= 1]
    similarite_non_explicite = [similarite for (idx_video, presence_termes_cle, freq_relative_termes_cle, similarite) \
                                in l_similarites_triees if presence_termes_cle < 1]
    # calculer le nombre de bins sur la similarité maximum (celle de la première vidéo)
    # (arrondir au montant sup)
    nb_bins = math.ceil(100 * l_similarites_triees[0][3])
    
    # tracer figure
    fig = go.Figure()
    fig.add_trace(go.Histogram(x=similarite_explicite, nbinsx=nb_bins, name="explicite", marker_color="red"))
    fig.add_trace(go.Histogram(x=similarite_non_explicite, nbinsx=nb_bins, name="non explicite", marker_color="green"))
    fig.update_layout(title=nature_similarite, barmode='stack', xaxis_title="similarité", yaxis_title="nb vidéos")
    fig.show()

In [ ]:
def montre_repartition_similarites_px(l_similarites_triees):
    
    # plotly express, il faut passer par un dataframe,
    # on a besoin de similarité et présence mot clé explicite ou non
    similarite = [similarite for (idx_video, presence_termes_cle, freq_relative_termes_cle, similarite) \
                  in l_similarites_triees]
    presence_explicite = ["Oui" if presence_termes_cle >= 1 else "Non" \
                for (idx_video, presence_termes_cle, freq_relative_termes_cle, similarite) in l_similarites_triees]
    dico_similarite = {'similarité': similarite, 'avec termes clé': presence_explicite}
    df_similarite = pd.DataFrame(dico_similarite)
    
    # calculer le nombre de bins sur la similarité maximum (celle de la première vidéo)
    # (arrondir au montant sup)
    nb_bins = math.ceil(100 * df_similarite.iloc[0,0])

    fig = px.histogram(df_similarite, x="similarité", color="avec termes clé",
                       color_discrete_map={'Oui':'red', 'Non':'green'},
                       nbins=nb_bins)
    fig.show()

In [ ]:
# 5.g
# représenter graphiquement la relation entre fréquence relative des mots clé dans les sous-titres 
# et similarité avec le texte de référenceh

def montre_relation_freq_termes_cle_similarite(l_similarites_triees):
    
    # plotly express, il faut passer par un dataframe,
    # on a besoin de similarité et fréquences relatives des termes clé
    # et uniquement sur les vidéos avec présence des termes clé
    similarite = [similarite for (idx_video, presence_termes_cle, freq_relative_termes_cle, similarite) \
                  in l_similarites_triees if presence_termes_cle >= 1]
    log_freq_relative = [math.log(freq_relative_termes_cle)\
                for (idx_video, presence_termes_cle, freq_relative_termes_cle, similarite) \
                in l_similarites_triees if presence_termes_cle >= 1]
    num_video = [idx_video for (idx_video, presence_termes_cle, freq_relative_termes_cle, similarite) \
                 in l_similarites_triees if presence_termes_cle >= 1]
    dict_similarite = {'num. video': num_video,
                       'similarite': similarite,
                       'log fréq. relative': log_freq_relative}
    df_similarite = pd.DataFrame(dict_similarite)
    
    fig = px.scatter(df_similarite, x="log fréq. relative", y="similarite")
    x_text = 0.5 * (max(log_freq_relative) + min(log_freq_relative))
    y_text = 1.25 * max(similarite)
    corr_similarite_freq_relative = pearsonr(log_freq_relative, similarite)
    text_corr = "Correlation %.2f" % (corr_similarite_freq_relative[0])
    fig.update_layout(autosize=False, width=600, height=400,
                      annotations=[go.layout.Annotation(x=x_text, y=y_text, text=text_corr, showarrow=False)])
    fig.show()

#### commencer donc avec les fréquences brutes

In [ ]:
# 6.a
l_similarites_freq = calcule_similarites_avec_ref(mat_doc_termes_freq_ss_titres,\
                                                  taille_corpus, taille_dico, df_videos)

In [ ]:
# 6.b
# afficher
affiche_videos_similaires(l_similarites_freq, 20)

vidéo 1161	 fréq. relative termes clé : 2.96	 similarité : 0.51
vidéo 1098	 fréq. relative termes clé : 3.65	 similarité : 0.50
vidéo 1202	 fréq. relative termes clé : 3.66	 similarité : 0.47
vidéo 894	 fréq. relative termes clé : 3.64	 similarité : 0.46
vidéo 909	 fréq. relative termes clé : 1.14	 similarité : 0.42
vidéo 940	 fréq. relative termes clé : 1.11	 similarité : 0.42
vidéo 976	 fréq. relative termes clé : 3.04	 similarité : 0.41
vidéo 993	 fréq. relative termes clé : 3.24	 similarité : 0.40
vidéo 1203	 fréq. relative termes clé : 5.33	 similarité : 0.40
vidéo 451	 fréq. relative termes clé : 6.17	 similarité : 0.40
vidéo 17	 fréq. relative termes clé : 2.61	 similarité : 0.39
vidéo 991	 fréq. relative termes clé : 2.95	 similarité : 0.39
vidéo 1097	 fréq. relative termes clé : 0.92	 similarité : 0.38
vidéo 963	 fréq. relative termes clé : 1.56	 similarité : 0.38
vidéo 1029	 fréq. relative termes clé : 0.41	 similarité : 0.37
vidéo 984	 fréq. relative termes clé : 2.12	 simil

In [ ]:
# 6.c
# graphiquement
montre_repartition_similarites(l_similarites_freq, "similarités comptages")

In [ ]:
# 6.c
# il peut être intéressant de revenir au sous-titres des vidéos les plus proches du texte de référence
print(df_videos.iloc[1098]['sous_titres'])

non et étudions ce scandale numéro 1 pi n'hésitez pas à commander bien sûr je fais des pauses pour pouvoir lire avec grand bonheur vos commentaires qui est le signe évidemment nous part de votre intérêt et puis surtout de votre participation c'est ça qui importe donc scandale un but on est en pénurie de masques c'est quand même incroyable encore une fois j'insiste mais faudrait que je vous remette le lien allez voir c'est l'émission complète un mois j'ai mis qu'un court extrait qui dont je suis content du succès mais il ya une émission d'une heure andré bercoff a interviewé christian perronne et là je peux vous dire que c'est sans langue de bois et c'est vraiment intéressant professeur peron qu'un jean paul et on a beaucoup beaucoup de ça à la radio sud radio reçoit énormément de gens et c'est vrai que le citoyen lambda il se dit que fait-on à votre avis comme ça les jeux je vous dis pas que vous n'allez pas en soi un homme politique mais quand on dit que fait-on qu'est ce que peut fai

In [ ]:
print(l_similarites_freq[0])

(1161, 1, 2.9642058165548097, 0.5124904921867591)


In [ ]:
# 6.d
# ainsi que la corrélation entre similarité et fréquence des termes clés
montre_relation_freq_termes_cle_similarite(l_similarites_freq)

#### Maintenant tf-idf, dans sa formule de base
- fréquence des termes : f (comptage absolu) => notation SMART : n
- fréquence inverse des documents (lissée) : n/N => notation SMART : t
- normalisation longueur des documents : cos => notation SMART : c (remarque, comme on calcule dessus le cosinus, c'est inutile)

In [ ]:
# 6.e
# paramétrage des métriques associées à tfidf
param_tfidf = 'ntc'

In [ ]:
# 6.f
# on utilise directement la fonction tf-idf de gensim
# il aurait été possible de créer nous même une telle fonction...
#tfidf = gensim.models.TfidfModel(dictionary=dico_termes_ss_titres, smartirs=None)
tfidf = gensim.models.TfidfModel(dictionary=dico_termes_ss_titres, smartirs=param_tfidf)
mat_doc_termes_tfidf_ss_titres = [tfidf[dico_termes_ss_titres.doc2bow(liste_tokens)] \
                                  for liste_tokens in mat_doc_tokens_ss_titres]

In [ ]:
# 6.g
l_similarites_tfidf = calcule_similarites_avec_ref(mat_doc_termes_tfidf_ss_titres,\
                                                   taille_corpus, taille_dico, df_videos)

In [ ]:
# 6.h
# afficher
affiche_videos_similaires(l_similarites_tfidf, 20)

vidéo 1161	 fréq. relative termes clé : 2.96	 similarité : 0.28
vidéo 1202	 fréq. relative termes clé : 3.66	 similarité : 0.23
vidéo 894	 fréq. relative termes clé : 3.64	 similarité : 0.23
vidéo 909	 fréq. relative termes clé : 1.14	 similarité : 0.21
vidéo 940	 fréq. relative termes clé : 1.11	 similarité : 0.21
vidéo 916	 fréq. relative termes clé : 0.44	 similarité : 0.19
vidéo 1108	 fréq. relative termes clé : 0.41	 similarité : 0.19
vidéo 1128	 fréq. relative termes clé : 0.96	 similarité : 0.18
vidéo 1029	 fréq. relative termes clé : 0.41	 similarité : 0.18
vidéo 1098	 fréq. relative termes clé : 3.65	 similarité : 0.18
vidéo 1097	 fréq. relative termes clé : 0.92	 similarité : 0.18
vidéo 660	 fréq. relative termes clé : 1.79	 similarité : 0.18
vidéo 1092	 fréq. relative termes clé : 2.06	 similarité : 0.18
vidéo 876	 fréq. relative termes clé : 1.66	 similarité : 0.17
vidéo 890	 fréq. relative termes clé : 1.42	 similarité : 0.17
vidéo 993	 fréq. relative termes clé : 3.24	 si

In [ ]:
# 6.i
# montrer
montre_repartition_similarites(l_similarites_tfidf, "similarités tf-idf")# 6.f

In [ ]:
# 6.j
# ainsi que la corrélation entre similarité et fréquence des termes clés
montre_relation_freq_termes_cle_similarite(l_similarites_tfidf)

##### Essayer avec différents paramètres :
##### Term frequency weighing:
b - binary, t or n - raw, a - augmented, l - logarithm, d - double logarithm, L - log average.
##### Document frequency weighting:
x or n - none, f - idf, t - zero-corrected idf, p - probabilistic idf.
##### Document normalization:
x or n - none, c - cosine, u - pivoted unique, b - pivoted character length.<p>
Par exemple 'afx', 'lfx', 'ltc'.<p>
Présence ou non (one-hot encoding), se traduit par 'bxx' (ou 'bnn')

In [ ]:
# 6.k
# mettre cela en fonction
def calcule_similarites_tfidf(dico_termes, mat_doc_tokens, param_tfidf,
                              taille_corpus, taille_dico, df_videos):

    # on utilise directement la fonction tf-idf de gensim
    # il aurait été possible de créer nous même une telle fonction...
    tfidf = gensim.models.TfidfModel(dictionary=dico_termes, smartirs=param_tfidf)
    mat_doc_termes_tfidf = [tfidf[dico_termes.doc2bow(liste_tokens)] \
                                  for liste_tokens in mat_doc_tokens]
    
    # calcul de la similarité
    l_similarites_tfidf = calcule_similarites_avec_ref(mat_doc_termes_tfidf,\
                            taille_corpus, taille_dico, df_videos)
    
    return l_similarites_tfidf

In [ ]:
# 6.l
# on crées une grille à partir de :
# fréquences de termes : n, a, l, L
# fréquence inverse de document : f, t, p
# on ne regarde que la seule similarité cosinus,
# équivalente (pour nous) à pas de norme puisqu'on calcule ensuite la distance cosinus
liste_tf = ['n', 'a', 'l', 'L']
liste_idf = ['f', 't', 'p']
liste_dn = ['c']

##### Prend un certain temps (10 minutes)

In [ ]:
# 6.m
dico_l_similarites_tfidf = {}

# boucler sur la grille
for tf in liste_tf:
    for idf in liste_idf:
        for dn in liste_dn:
            param_tfidf = tf + idf + dn
            print(param_tfidf)
            
            dico_l_similarites_tfidf[param_tfidf] = \
            calcule_similarites_tfidf(dico_termes_ss_titres, \
                                      mat_doc_tokens_ss_titres, \
                                      param_tfidf, \
                                      taille_corpus, taille_dico, \
                                      df_videos)

nfc
ntc
npc
afc
atc
apc
lfc
ltc
lpc
Lfc
Ltc
Lpc


In [ ]:
# 6.n
# fonction renvoyant la liste de toutes les vidéos (tous les index)
# autant ou plus similaires à la vidéo de référence
# qu'une proportion prise sur l'ensemble des vidéos 
# contenant explicitement les termes clés 

# on renvoit une liste de 0 ou de 1 selon les index vidéos
def liste_videos_au_top_explicite(l_similarites_triees, 
                                  proportion_termes_cle_explicite,
                                  taille_corpus,
                                  taille_corpus_avec_termes_cle):
    l_videos_top = [0] * (taille_corpus-1)
    idx_termes_cle_explicite = 0
    for (idx_video, presence_termes_cle, freq_relative_termes_cle, similarite) \
        in l_similarites_triees:
        if presence_termes_cle >= 1:
            idx_termes_cle_explicite += 1
        l_videos_top[idx_video] = 1
        if idx_termes_cle_explicite > \
           proportion_termes_cle_explicite * taille_corpus_avec_termes_cle:
            return l_videos_top

In [ ]:
# 6.o
# comme critères d'évaluation pour retenir la meilleure combinaison, on va :
# - considérer la corrélation entre les similarités et 
# les fréquences relatives des termes clés
# - considérer le profil de proportions de vidéos avec mots clés et sans mots clés
# dans les n % vidéos les plus similaires avec la vidéo de référence
proportions_termes_cle_explicite = [0.50, 0.70, 0.80, 0.90]

# boucler sur les paramétrages tf-idf
for tf_idf_dn in dico_l_similarites_tfidf:
    
    print(tf_idf_dn, " :")
    l_similarites_triees = dico_l_similarites_tfidf[tf_idf_dn]
    
    # calcul des corrélations
    similarite = [similarite for (idx_video, presence_termes_cle, freq_relative_termes_cle, similarite) \
                  in l_similarites_triees if presence_termes_cle >= 1]
    log_freq_relative = [math.log(freq_relative_termes_cle)\
                for (idx_video, presence_termes_cle, freq_relative_termes_cle, similarite) \
                in l_similarites_triees if presence_termes_cle >= 1]
    
    corr_similarite_freq_relative = pearsonr(log_freq_relative, similarite)
    print("corrélation : %.3f" % (corr_similarite_freq_relative[0]))
    
    # calcul des 
    for proportion_termes_cle_explicite in proportions_termes_cle_explicite:
        liste_videos_top = liste_videos_au_top_explicite(l_similarites_triees, 
                                                         proportion_termes_cle_explicite,
                                                         taille_corpus,
                                                         taille_corpus_avec_termes_cle)
        nb_videos_top = sum(liste_videos_top)
        print("pour obtenir %d %% des vidéos explicitement avec termes clé" % \
              (100 * proportion_termes_cle_explicite))
        print("\til faut en tout %2.0f %% des vidéos, dont %2.0f %% avec termes clé" % \
              (100 * nb_videos_top / (taille_corpus-1),\
               100 * proportion_termes_cle_explicite * taille_corpus_avec_termes_cle / nb_videos_top))
    
    print("---------------------------")

nfc  :
corrélation : 0.389
pour obtenir 50 % des vidéos explicitement avec termes clé
	il faut en tout  7 % des vidéos, dont 95 % avec termes clé
pour obtenir 70 % des vidéos explicitement avec termes clé
	il faut en tout 11 % des vidéos, dont 85 % avec termes clé
pour obtenir 80 % des vidéos explicitement avec termes clé
	il faut en tout 14 % des vidéos, dont 76 % avec termes clé
pour obtenir 90 % des vidéos explicitement avec termes clé
	il faut en tout 18 % des vidéos, dont 64 % avec termes clé
---------------------------
ntc  :
corrélation : 0.389
pour obtenir 50 % des vidéos explicitement avec termes clé
	il faut en tout  7 % des vidéos, dont 95 % avec termes clé
pour obtenir 70 % des vidéos explicitement avec termes clé
	il faut en tout 11 % des vidéos, dont 85 % avec termes clé
pour obtenir 80 % des vidéos explicitement avec termes clé
	il faut en tout 14 % des vidéos, dont 76 % avec termes clé
pour obtenir 90 % des vidéos explicitement avec termes clé
	il faut en tout 18 % des 

#### Manifestement, le meilleur choix est celui des paramètres 'npc'

#### Sélection des vidéos les plus similaires
Nous prenons l'union de :<p>
- des n1 % de l'ensemble des vidéos les mieux classées<p>
- des n2 % de l'ensemble des vidéos explicitement avec termes clé<p>
n1 et n2 se voient d'après les profils affichés plus haut<p>
Par exemple dans l'exemple gilet jaune, on prend les 10 % des vidéos les mieux classées, à compléter, par les vidéos explicitement gilet jaune nécessaires pour arriver à 80 % de couverure


In [ ]:
# 7.a
# fonction renvoyant la liste de toutes les vidéos (tous les index)
# autant ou plus similaires à la vidéo de référence
# qu'une proportion prise sur l'ensemble des vidéos 
# ou bien dans le top pourcentage des vidéos contenant les mots clés

# typiquement proportion_top_similarite est un petit nombre (10 %)
# proportion_termes_cle_explicite un grand nombre (80 %)

# on renvoit une liste de 0 ou de 1 selon les index vidéos
def selectionne_top_videos(l_similarites_triees,
                           proportion_top_similarite,
                           proportion_termes_cle_explicite,
                           taille_corpus,
                           taille_corpus_avec_termes_cle):
    
    l_videos_top = [0] * (taille_corpus-1)
    idx_termes_cle_explicite = 0
    top_similarite_atteint = False
    termes_cle_explicite_atteint = False
    for (idx_video, presence_termes_cle, freq_relative_termes_cle, similarite) \
        in l_similarites_triees:
        
        # on est en présence d'une vidéo avec les termes clés
        if presence_termes_cle >= 1:
            idx_termes_cle_explicite += 1
        
        # on a pris les vidéos les plus similaires qu'on voulait
        if idx_video > proportion_top_similarite * (taille_corpus-1):
            top_similarite_atteint = True
        # on a pris les vidéos abec termes clé qu'on voulait
        if idx_termes_cle_explicite > proportion_termes_cle_explicite * taille_corpus_avec_termes_cle:
            termes_cle_explicite_atteint = True
        
        # toper les vidéos qu'on veut conserver
        if presence_termes_cle >= 1 and termes_cle_explicite_atteint == False:
            l_videos_top[idx_video] = 1
        if top_similarite_atteint == False:
            l_videos_top[idx_video] = 1
        
    return l_videos_top
    

#### CHANGER LES VALEURS DE PARAMETRE SELON BESOINS

In [ ]:
# 7.b
# sélection avec les paramètres vus d'après l'exécution précédente
proportion_top_similarite = 0.10
proportion_termes_cle_explicite = 0.80

In [ ]:
# 7.c
l_similarites_triees = dico_l_similarites_tfidf['npc']
liste_videos_top = selectionne_top_videos(l_similarites_triees, 
                                          proportion_top_similarite,
                                          proportion_termes_cle_explicite,
                                          taille_corpus,
                                          taille_corpus_avec_termes_cle)
nb_videos_top = sum(liste_videos_top)
print(nb_videos_top)

560


#### Ecriture du fichier thématique

#### A CHANGER OU NON SELON LE THEME COURANT

In [ ]:
# 7.d
CSV_ECHANTILLON_THEMATIQUE = CSV_ECHANTILLON_THEMATIQUE_GJ

In [ ]:
# 7.e
# à partir des index sélectionnés (mis à 1 dans la liste l_videos_top)
# créer le dataframe des tops vidéos
df_videos['est_top'] = liste_videos_top
df_top_videos = df_videos.loc[df_videos['est_top'] == 1]
print(len(df_top_videos))

560


In [ ]:
# 7.f
df_top_videos.to_csv(CSV_ECHANTILLON_THEMATIQUE, sep=',', encoding='utf-8', header=True,
                     index=False, quoting=csv.QUOTE_ALL)